In [2]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('vitamins.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
forecast_df.to_excel('vitamins_results.xlsx', index=False)

Epoch 1/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 1s 435us/step - loss: 541.3549 - val_loss: 321.3840
Epoch 2/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - loss: 256.8912 - val_loss: 304.4663
Epoch 3/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - loss: 313.8696 - val_loss: 307.6487
Epoch 4/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - loss: 229.6772 - val_loss: 304.2637
Epoch 5/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step - loss: 287.8879 - val_loss: 299.8530
Epoch 6/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - loss: 243.4575 - val_loss: 307.7798
Epoch 7/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - loss: 268.0094 - val_loss: 287.4624
Epoch 8/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step - loss: 246.0819 - val_loss: 294.3070
Epoch 9/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - loss: 248.7552 - val_loss: 291.6401
Epoch 10/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 381us/step - loss: 263.1522 - val_loss: 302.6482
Epoch 11/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 244.3

517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - loss: 206.6372 - val_loss: 270.6751
Epoch 66/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - loss: 210.3747 - val_loss: 297.5478
Epoch 67/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - loss: 219.6460 - val_loss: 279.3168
Epoch 68/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - loss: 265.7833 - val_loss: 276.9785
Epoch 69/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - loss: 191.7286 - val_loss: 270.5515
Epoch 70/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 188.9298 - val_loss: 278.1007
Epoch 71/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - loss: 206.9813 - val_loss: 274.4869
Epoch 72/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - loss: 209.3078 - val_loss: 272.3492
Epoch 73/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step - loss: 194.4067 - val_loss: 282.1191
Epoch 74/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - loss: 214.0895 - val_loss: 288.9990
Epoch 75/100
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - loss: 201.7934 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1